In [38]:
### Set values here ###
config = 'configs/zero_shot/train/k400/16_16_vifi_clip.yaml'
output_folder_name = "exp"
pretrained_model_path = "/content/drive/MyDrive/Colabs/CLIP/vifi_clip_10_epochs_k400_full_finetuned.pth"
# List the action names for which ViFi-CLIP will perform action recognition
class_names = ['dancing', 'drum beating', 'swimming', "climbing stairs"]
# Load your video example:
video_path = '/content/drive/MyDrive/Colabs/CLIP/AMADEUS_eat_u_nm_np1_fr_goo_10.avi'

In [3]:
# !git clone https://github.com/muzairkhattak/ViFi-CLIP.git

Cloning into 'ViFi-CLIP'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 265 (delta 16), reused 14 (delta 6), pack-reused 227
Receiving objects: 100% (265/265), 23.77 MiB | 24.68 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [ ]:
#!pip install -r requirements.txt
# !pip3 install openmim # if mmcv cannot be installed
# !mim install mmcv-full

In [2]:
%cd ViFi-CLIP

/content/ViFi-CLIP


In [3]:
%ls

clip/      datasets_splits/  LICENSE    requirements.txt  ViFi-CLIP_Inference_custom_video.ipynb
configs/   docs/             main.py    trainers/
datasets/  labels/           README.md  utils/


In [8]:
# !pip install -r requirements.txt

In [4]:
import torch
import torch.nn as nn
from utils.config import get_config
from utils.logger import create_logger
import time
import numpy as np
from utils.config import get_config
from trainers import vificlip
from datasets.pipeline import *

/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [39]:
# Step 1:
# Configuration class
class parse_option():
    def __init__(self):
        self.config = config
        self.output =  output_folder_name   # Name of output folder to store logs and save weights
        self.resume = pretrained_model_path
        # No need to change below args.
        self.only_test = True
        self.opts = None
        self.batch_size = None
        self.pretrained = None
        self.accumulation_steps = None
        self.local_rank = 0
args = parse_option()
config = get_config(args)
# logger

=> merge config from configs/zero_shot/train/k400/16_16_vifi_clip.yaml


In [7]:
!mkdir exp

In [40]:
logger = create_logger(output_dir=args.output, name=f"{config.MODEL.ARCH}")
print(2)
logger.info(f"working dir: {config.OUTPUT}")

2
[2024-07-18 06:44:46 ViT-B/16](<ipython-input-40-c63b87ca9809> 3): INFO working dir: exp
[2024-07-18 06:44:46 ViT-B/16](<ipython-input-40-c63b87ca9809> 3): INFO working dir: exp
[2024-07-18 06:44:46 ViT-B/16](<ipython-input-40-c63b87ca9809> 3): INFO working dir: exp


In [41]:
# Step 2:
# Create the ViFi-CLIP models and load pretrained weights
model = vificlip.returnCLIP(config,
                            logger=logger,
                            class_names=class_names,)
model = model.float().cuda()  # changing to cuda here

[2024-07-18 06:44:48 ViT-B/16](vificlip.py 202): INFO Loading CLIP (backbone: ViT-B/16)
[2024-07-18 06:44:48 ViT-B/16](vificlip.py 202): INFO Loading CLIP (backbone: ViT-B/16)
[2024-07-18 06:44:48 ViT-B/16](vificlip.py 202): INFO Loading CLIP (backbone: ViT-B/16)
[2024-07-18 06:44:53 ViT-B/16](vificlip.py 205): INFO Building ViFi-CLIP CLIP
[2024-07-18 06:44:53 ViT-B/16](vificlip.py 205): INFO Building ViFi-CLIP CLIP
[2024-07-18 06:44:53 ViT-B/16](vificlip.py 205): INFO Building ViFi-CLIP CLIP
[2024-07-18 06:44:53 ViT-B/16](vificlip.py 222): INFO Turning on gradients for COMPLETE ViFi-CLIP model
[2024-07-18 06:44:53 ViT-B/16](vificlip.py 222): INFO Turning on gradients for COMPLETE ViFi-CLIP model
[2024-07-18 06:44:53 ViT-B/16](vificlip.py 222): INFO Turning on gradients for COMPLETE ViFi-CLIP model
[2024-07-18 06:44:53 ViT-B/16](vificlip.py 245): INFO Parameters to be updated: {'image_encoder.transformer.resblocks.3.mlp.c_fc.bias', 'image_encoder.transformer.resblocks.8.attn.in_proj_bi

In [31]:
logger.info(f"==============> Resuming form {config.MODEL.RESUME}....................")
print(config.MODEL.RESUME)
checkpoint = torch.load(config.MODEL.RESUME, map_location='cpu')
load_state_dict = checkpoint['model']
# now remove the unwanted keys:
if "module.prompt_learner.token_prefix" in load_state_dict:
    del load_state_dict["module.prompt_learner.token_prefix"]

if "module.prompt_learner.token_suffix" in load_state_dict:
    del load_state_dict["module.prompt_learner.token_suffix"]

if "module.prompt_learner.complete_text_embeddings" in load_state_dict:
    del load_state_dict["module.prompt_learner.complete_text_embeddings"]
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in load_state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

[2024-07-18 06:35:47 ViT-B/16](<ipython-input-31-8d5d4db1129f> 1): INFO ==============> Resuming form /content/drive/MyDrive/Colabs/CLIP/vifi_clip_10_epochs_k400_full_finetuned.pth....................
[2024-07-18 06:35:47 ViT-B/16](<ipython-input-31-8d5d4db1129f> 1): INFO ==============> Resuming form /content/drive/MyDrive/Colabs/CLIP/vifi_clip_10_epochs_k400_full_finetuned.pth....................
[2024-07-18 06:35:47 ViT-B/16](<ipython-input-31-8d5d4db1129f> 1): INFO ==============> Resuming form /content/drive/MyDrive/Colabs/CLIP/vifi_clip_10_epochs_k400_full_finetuned.pth....................
/content/drive/MyDrive/Colabs/CLIP/vifi_clip_10_epochs_k400_full_finetuned.pth


In [42]:
# load params
msg = model.load_state_dict(new_state_dict, strict=False)
logger.info(f"resume model: {msg}")

[2024-07-18 06:44:56 ViT-B/16](<ipython-input-42-09d8243ad764> 3): INFO resume model: _IncompatibleKeys(missing_keys=['prompt_learner.complete_text_embeddings'], unexpected_keys=[])
[2024-07-18 06:44:56 ViT-B/16](<ipython-input-42-09d8243ad764> 3): INFO resume model: _IncompatibleKeys(missing_keys=['prompt_learner.complete_text_embeddings'], unexpected_keys=[])
[2024-07-18 06:44:56 ViT-B/16](<ipython-input-42-09d8243ad764> 3): INFO resume model: _IncompatibleKeys(missing_keys=['prompt_learner.complete_text_embeddings'], unexpected_keys=[])


In [53]:
# Step 3:
# Preprocessing for video
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_bgr=False)
scale_resize = int(256 / 224 * config.DATA.INPUT_SIZE)
val_pipeline = [
    dict(type='DecordInit'),
    dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=config.DATA.NUM_FRAMES, test_mode=True),
    dict(type='DecordDecode'),
    dict(type='Resize', scale=(-1, scale_resize)),
    dict(type='CenterCrop', crop_size=config.DATA.INPUT_SIZE),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='FormatShape', input_format='NCHW'),
    dict(type='Collect', keys=['imgs'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs'])
]
if config.TEST.NUM_CROP == 3:
    val_pipeline[3] = dict(type='Resize', scale=(-1, config.DATA.INPUT_SIZE))
    val_pipeline[4] = dict(type='ThreeCrop', crop_size=config.DATA.INPUT_SIZE)
if config.TEST.NUM_CLIP > 1:
    val_pipeline[1] = dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=config.DATA.NUM_FRAMES, multiview=config.TEST.NUM_CLIP)
pipeline = Compose(val_pipeline)

In [54]:
dict_file = {'filename': video_path, 'tar': False, 'modality': 'RGB', 'start_index': 0}

In [36]:
# !pip install decord

  Using cached decord-0.6.0-py3-none-manylinux2010_x86_64.whl (13.6 MB)


In [56]:
np.int = np.int64

In [57]:
video = pipeline(dict_file)
video_tensor = video['imgs'].unsqueeze(0).cuda().float()
# Inference through ViFi-CLIP
with torch.no_grad():
    with torch.cuda.amp.autocast():
        logits = model(video_tensor)
pred_index = logits.argmax(-1)

In [58]:
print(f'logits: {logits}')
print(f'predicted action category is : {class_names[pred_index]}')

logits: tensor([[10.5938,  9.2500,  8.1172, 10.5000]], device='cuda:0',
       dtype=torch.float16)
predicted action category is : dancing
